**Define all the imports**

In [1]:
import os
import cv2
import numpy as np

from torchvision import transforms as T
import torch

import matplotlib.pyplot as plt

import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

**Apple Dataset**

In [2]:
class Apple_Dataset(object):
    def __init__(self, root_dir, transforms):
        self.root_dir = root_dir
        self.transforms = transforms

        # Load all image and mask files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root_dir, "masks"))))

    def __getitem__(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        mask_path = os.path.join(self.root_dir, "masks", self.masks[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))
        mask = cv2.imread(mask_path, 0)

        obj_ids = np.unique(mask)

        # Remove Background
        obj_ids = obj_ids[1:]

        # Split the color-encoded masks into a set of binary masks
        masks = mask == obj_ids[:, None, None]

        # Get bbox coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        h, w = mask.shape
        for each in range(num_objs):
            pos = np.where(masks[each])
            xmin = np.min(pos[1])//2
            xmax = np.max(pos[1])//2
            ymin = np.min(pos[0])//2
            ymax = np.max(pos[0])//2

            if xmin == xmax or ymin == ymax:
                continue

            xmin = np.clip(xmin, a_min=0, a_max=w)
            xmax = np.clip(xmax, a_min=0, a_max=w)
            ymin = np.clip(ymin, a_min=0, a_max=h)
            ymax = np.clip(ymax, a_min=0, a_max=h)
            boxes.append([xmin, ymin, xmax, ymax])

        # Convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # There is only one class (apples)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # All instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

#         if self.transforms is not None:
#             img, target = self.transforms(img, target)

        img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)

**Utils**

In [3]:
class Utils(object):
    def __init__(self):
        pass
    
    def get_transform(self, train):
        transforms = [T.ToTensor()]
        # if train:
        #     transforms.append(T.RandomHorizontalFlip(0.5))
        return T.Compose(transforms)


    def my_collate_fn(self, batch):
        images = [each[0] for each in batch]
        targets = [each[1] for each in batch]
        return images, targets


    def generate_bbox(self, img, boxes):
        for (x_min, y_min, x_max, y_max) in boxes:
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            
    def get_frcnn_model_instance(self, num_classes):
        # load an instance segmentation model pre-trained pre-trained on COCO
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model
    

**Test Dataset**

In [4]:
# root_path = '../input/minneapple/Dataset/Dataset'


# util = Utils()
# train_dataset = Apple_Dataset(os.path.join(root_path, 'train'), util.get_transform(True))
# val_dataset = Apple_Dataset(os.path.join(root_path, 'val'), util.get_transform(True))

# data_loader = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=util.my_collate_fn),
#                'val': torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=util.my_collate_fn)}

# for i, (images, targets) in enumerate(data_loader['train']):
#     img = images[0]
#     print(img.shape)
#     util.generate_bbox(img, targets[0]['boxes'])
#     plt.figure()
#     plt.figure(figsize=(10, 10))
#     plt.subplot(121)
#     plt.imshow(img)
#     break

**Training Using Faster RCNN**

In [5]:
root_path = '../input/minneapple/Dataset/Dataset'
out_folder = '/kaggle/working/apple_counting/'
num_epochs = 50

util = Utils()
train_dataset = Apple_Dataset(os.path.join(root_path, 'train'), util.get_transform(True))
val_dataset = Apple_Dataset(os.path.join(root_path, 'val'), util.get_transform(True))

data_loader = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=util.my_collate_fn),
               'val': torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=util.my_collate_fn)}

model = util.get_frcnn_model_instance(num_classes=2)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 40], gamma=0.1)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = model.to(device)

if not os.path.exists(out_folder + "/Models/"):
    os.makedirs(out_folder + "/Models/")

train_loss = []
val_loss = []
for epoch in range(num_epochs):
    for category in ['train', 'val']:
        loss_epoch = []
        for i, (images, targets) in enumerate(data_loader[category]):
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            model.train()
            
            if category == 'train':
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

                optimizer.zero_grad()
                losses.backward()
                optimizer.step()
                
                
            else:
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
                
            
            loss_epoch.append(losses.item())
        
        avg_loss = sum(loss_epoch)/len(loss_epoch)
            
        if category == 'train':
            train_loss.append(avg_loss)
        else:
            val_loss.append(avg_loss)
            if avg_loss <= min(val_loss):
                torch.save(model.state_dict(), out_folder + "/Models/FRCNN_30Dec2020_best.pt".format(epoch))
        
        print ("{} Loss at epoch {} of {}: {}".format(category, epoch, num_epochs, avg_loss))
    lr_scheduler.step()

print(val_loss)
print(train_loss)
torch.save(model.state_dict(), out_folder + "/Models/FRCNN_30Dec2020_{}.pt".format(epoch))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3000: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and uses scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/opt/conda/lib/python3.7/site-packages/torchvision/ops/boxes.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = keep.nonzero().squeeze(1)


train Loss at epoch 0 of 50: 1.5362044879705599
val Loss at epoch 0 of 50: 1.3931080137982088
train Loss at epoch 1 of 50: 1.2813068480774907
val Loss at epoch 1 of 50: 1.2039647961364073
train Loss at epoch 2 of 50: 1.1069933801594347
val Loss at epoch 2 of 50: 1.0485957931069767
train Loss at epoch 3 of 50: 1.0720789462032885
val Loss at epoch 3 of 50: 1.060794744421454
train Loss at epoch 4 of 50: 0.9852072826706537
val Loss at epoch 4 of 50: 0.9793551792116726
train Loss at epoch 5 of 50: 0.9688308504548403
val Loss at epoch 5 of 50: 0.9655728743356817
train Loss at epoch 6 of 50: 0.9265174125090684
val Loss at epoch 6 of 50: 0.9395097914864036
train Loss at epoch 7 of 50: 0.8903940414438153
val Loss at epoch 7 of 50: 0.9252334359814139
train Loss at epoch 8 of 50: 0.8628484358881959
val Loss at epoch 8 of 50: 0.9074215257869047
train Loss at epoch 9 of 50: 0.843501877076555
val Loss at epoch 9 of 50: 0.900152762146557
train Loss at epoch 10 of 50: 0.7930458981801968
val Loss at ep

**Code Test**

In [6]:
def draw_bounding_box(img, boxes, scores):
    boxes = np.array(boxes, dtype=int)
    count = 0
    for box, score in zip(boxes, scores):
        if score > 0.9:
            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 4)
            count += 1

    return count

model.eval()

sample_img = '../input/minneapple/Dataset/Dataset/test/images/20150919_174151_image1.png'

img1 = cv2.imread(sample_img)
img = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))


trans = util.get_transform(True)
img = trans(img)
img = img.to(device)

y = model([img])

boxes = y[0]['boxes'].cpu().detach().numpy()
scores = y[0]['scores'].cpu().detach().numpy()

num_apples = draw_bounding_box(img1, boxes, scores)
print(num_apples)

plt.figure()
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(img1)

# mask = np.array(y.cpu().detach().data[0].permute(1,2,0) *255, dtype = np.uint8)
# mask = cv2.resize(mask, (720, 1280))


# plt.subplot(122)
# plt.imshow(mask)
# plt.show()

# root_path = '../input/minneapple/detection/train'

# util = Utils()

# dataset = Apple_Dataset(root_path, util.get_transform(True))
# data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=4,
#                                               collate_fn=util.my_collate_fn)

# for i, (images, targets) in enumerate(data_loader):
#     if i > 0:
#         break
#     img = np.array(images[0].permute(1, 2, 0).data)
    
#     util.generate_bbox(img, targets[0]['boxes'])
#     plt.figure()
#     plt.figure(figsize=(30, 30))
#     plt.imshow(img)
#     plt.show()    

50


<Figure size 432x288 with 0 Axes>